In [1]:
import sys
sys.path.append('../')

import canopus
import os
import pytket.qasm
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_dag
from accel_utils import fuzzy_equal
import numpy as np

A circuit snippet

In [3]:
qc = QuantumCircuit(5)
qc.cx(0, 1)
qc.cx(3, 2)
qc.cx(2, 1)
qc.cx(3, 4)
qc.draw()

qc_can = canopus.rebase_to_tk2(qc)
qc_can.swap(0, 1)

print(canopus.SynthCostEstimator('cx').eval_circuit_cost(qc_can))

qc_can.draw(fold=200)


4.0


┌───────────────┐  ┌───────────────┐  ┌────────────┐                                        
q_0: ──┤ U(7π/2,π,π/2) ├──┤0              ├──┤ U(π/2,0,π) ├──────────────────────────────────────X─
       ├───────────────┴┐ │  Can(0.5,0,0) │  └────────────┘ ┌───────────────┐ ┌────────────────┐ │ 
q_1: ──┤ U(π,-π/2,7π/2) ├─┤1              ├─────────────────┤0              ├─┤ U(0,-π/2,3π/2) ├─X─
     ┌─┴────────────────┴┐├───────────────┤ ┌──────────────┐│  Can(0.5,0,0) │ └─┬────────────┬─┘   
q_2: ┤ U(3π/2,-π/2,3π/2) ├┤0              ├─┤ U(π/2,0,π/2) ├┤1              ├───┤ U(π/2,0,π) ├─────
     └──┬─────────────┬──┘│  Can(0.5,0,0) │┌┴──────────────┤└─┬────────────┬┘   └────────────┘     
q_3: ───┤ U(π/2,0,4π) ├───┤1              ├┤0              ├──┤ U(π/2,0,π) ├───────────────────────
     ┌──┴─────────────┴──┐└───────────────┘│  Can(0.5,0,0) │┌─┴────────────┴─┐                     
q_4: ┤ U(3π/2,-π/2,3π/2) ├─────────────────┤1              ├┤ U(0,-π/2,3π/2) ├─────────────────────
     └───────────────────┘                 └───────────────┘└────────────────┘

In [3]:
canopus.rebase_to_canonical(qc).draw(fold=200)

global phase: π
            ┌────────────┐      ┌───────────────┐   ┌───────────────┐                                           
q_0: ───────┤ U(π/2,π,0) ├──────┤1              ├───┤ U(π/2,-π/2,0) ├───────────────────────────────────────────
       ┌────┴────────────┴───┐  │  Can(0.5,0,0) │┌──┴───────────────┴──┐┌───────────────┐┌─────────────────────┐
q_1: ──┤ U(0.10111,π/2,-π/2) ├──┤0              ├┤ U(1.6713,-3π/2,π/2) ├┤1              ├┤ U(1.5714,-π/2,-π/2) ├
     ┌─┴─────────────────────┴─┐├───────────────┤└┬───────────────────┬┘│  Can(0.5,0,0) │└──┬────────────────┬─┘
q_2: ┤ U(0.00055691,-π/2,-π/2) ├┤1              ├─┤ U(π/2,-7.8545,-π) ├─┤0              ├───┤ U(π/2,3π/2,-π) ├──
     └─────┬──────────────┬────┘│  Can(0.5,0,0) │ └┬──────────────────┤ ├───────────────┤   ├───────────────┬┘  
q_3: ──────┤ U(π/2,-2π,0) ├─────┤0              ├──┤ U(π/2,π/2,-3π/2) ├─┤1              ├───┤ U(π/2,-π/2,0) ├───
       ┌───┴──────────────┴──┐  └───────────────┘  └──────────────────┘ │  Can(0.5,0,0) │┌──┴───────────────┴──┐
q_4: ──┤ U(0.10111,π/2,-π/2) ├──────────────────────────────────────────┤0              ├┤ U(1.6719,-5π/2,π/2) ├
       └─────────────────────┘                                          └───────────────┘└─────────────────────┘

Real-world program

In [28]:
input_dpath = './output/logical/tk2'
print(os.listdir(input_dpath))

['ising_n26.qasm', 'cat_state_n22.qasm', 'ghz_state_n23.qasm', 'wstate_n27.qasm', 'qft_n18.qasm', 'dnn_n16.qasm', 'qram_n20.qasm', 'qec9xz_n17.qasm', 'qpeexact_16.qasm', 'knn_n25.qasm', 'gcm_n13.qasm', 'swap_test_n25.qasm', 'multiplier_n15.qasm', 'bv_n19.qasm', 'qf21_n15.qasm', 'bigadder_n18.qasm', 'sat_n11.qasm']


In [ ]:
circ = pytket.qasm.circuit_from_qasm(os.path.join(input_dpath, 'qec9xz_n17.qasm'))
qc = canopus.utils.tket_to_qiskit(circ)
canopus.utils.print_circ_info(qc)

NameError: name 'pytket' is not defined

In [37]:
qc.draw(fold=10000)

┌───────────────┐  ┌───────────────┐┌───────────────┐   ┌──────────────┐  ┌───────────────┐┌───────────────┐                                 ┌───────────────┐┌───────────────┐                                                                                      ┌───────────────┐                                                                                                                                                                                                                                                                                                                       
 q_0: ──┤ U(π,3π/2,π/2) ├──┤0              ├┤0              ├───┤ U(π/2,0,π/2) ├──┤0              ├┤0              ├─────────────────────────────────┤0              ├┤ U(7π/2,π,π/2) ├──────────────────────────────────────────────────────────────────────────────────────┤0              ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      ┌─┴───────────────┴─┐│               ││               │   └──────────────┘  │  Can(0.5,0,0) ││               │ ┌────────────┐                  │               │└───────────────┘                 ┌───────────────┐                                                    │               │┌───────────────┐ ┌───────────────┐                                                                                      ┌───────────────┐                                                                                                                                                                             
 q_1: ┤ U(3π/2,3π/2,3π/2) ├┤               ├┤               ├─────────────────────┤1              ├┤  Can(0.5,0,0) ├─┤ U(π/2,0,π) ├──────────────────┤               ├──────────────────────────────────┤0              ├────────────────────────────────────────────────────┤               ├┤0              ├─┤ U(7π/2,π,π/2) ├──────────────────────────────────────────────────────────────────────────────────────┤0              ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      ├───────────────────┤│  Can(0.5,0,0) ││               │                     └───────────────┘│               │┌┴────────────┴┐                 │               │                                  │               │                                                    │               ││               │ └───────────────┘                                   ┌───────────────┐┌───────────────┐                 │               │                                   ┌───────────────┐                                                                                                                         
 q_2: ┤ U(3π/2,3π/2,3π/2) ├┤               ├┤               ├──────────────────────────────────────┤1              ├┤ U(π/2,0,π/2) ├─────────────────┤               ├──────────────────────────────────┤               ├────────────────────────────────────────────────────┤               ├┤               ├─────────────────────────────────────────────────────┤0              ├┤ U(7π/2,π,π/2) ├─────────────────┤               ├───────────────────────────────────┤0              ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      ├───────────────────┤│               ││               │┌───────────────────┐┌───────────────┐├───────────────┤└──────────────┘┌───────────────┐│               │  ┌────────────┐                  │               │ ┌───────────────┐                                  │               ││               │                                                     │               │└───────────────┘                 │            

In [43]:
list(dag.reverse_ops().op_nodes())

[DAGOpNode(op=Instruction(name='u', num_qubits=1, num_clbits=0, params=[0.0, 4.71238898038469, 4.71238898038469]), qargs=(Qubit(QuantumRegister(17, 'q'), 16),), cargs=()),
 DAGOpNode(op=Instruction(name='u', num_qubits=1, num_clbits=0, params=[0.0, 4.71238898038469, 4.71238898038469]), qargs=(Qubit(QuantumRegister(17, 'q'), 15),), cargs=()),
 DAGOpNode(op=Instruction(name='u', num_qubits=1, num_clbits=0, params=[1.5707963267948966, 0.0, 3.141592653589793]), qargs=(Qubit(QuantumRegister(17, 'q'), 8),), cargs=()),
 DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.5, 0.0, 0.0]), qargs=(Qubit(QuantumRegister(17, 'q'), 8), Qubit(QuantumRegister(17, 'q'), 16)), cargs=()),
 DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.5, 0.0, 0.0]), qargs=(Qubit(QuantumRegister(17, 'q'), 7), Qubit(QuantumRegister(17, 'q'), 16)), cargs=()),
 DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.5, 0.0, 0.0]), qargs=(Qubit(QuantumRegis

In [38]:
dag = circuit_to_dag(qc)

In [39]:
commutative_pairs = []
for node in dag.op_nodes():
    if not (isinstance(node.op, canopus.CanonicalGate) and fuzzy_equal(node.op.params[1], 0) and fuzzy_equal(node.op.params[2], 0)):
        continue
    for pred in dag.op_predecessors(node):
        if isinstance(pred.op, canopus.CanonicalGate) and fuzzy_equal(pred.op.params[1], 0) and fuzzy_equal(pred.op.params[2], 0):
            if set((node, pred)) not in commutative_pairs:
                commutative_pairs.append(set((node, pred)))


In [40]:
len(commutative_pairs)

29

In [41]:
commutative_pairs

[{DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.5, 0.0, 0.0]), qargs=(Qubit(QuantumRegister(17, 'q'), 0), Qubit(QuantumRegister(17, 'q'), 3)), cargs=()),
  DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.5, 0.0, 0.0]), qargs=(Qubit(QuantumRegister(17, 'q'), 0), Qubit(QuantumRegister(17, 'q'), 6)), cargs=())},
 {DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.5, 0.0, 0.0]), qargs=(Qubit(QuantumRegister(17, 'q'), 3), Qubit(QuantumRegister(17, 'q'), 4)), cargs=()),
  DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.5, 0.0, 0.0]), qargs=(Qubit(QuantumRegister(17, 'q'), 3), Qubit(QuantumRegister(17, 'q'), 5)), cargs=())},
 {DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.5, 0.0, 0.0]), qargs=(Qubit(QuantumRegister(17, 'q'), 0), Qubit(QuantumRegister(17, 'q'), 1)), cargs=()),
  DAGOpNode(op=Instruction(name='can', num_qubits=2, num_clbits=0, params=[0.5, 0.0